### Importing the Required Libraries to setup the connection from Local to Snowflake

In [ ]:
%pip install snowflake-connector-python[pandas]

%pip install dotenv

In [ ]:
import snowflake.connector
import os
from dotenv import load_dotenv

# Load the variables from .env
load_dotenv()

# Establish the connection
try:
    conn = snowflake.connector.connect(
        user=os.getenv('SNOWFLAKE_USER'),
        password=os.getenv('SNOWFLAKE_PASSWORD'),
        account=os.getenv('SNOWFLAKE_ACCOUNT'),
        warehouse=os.getenv('SNOWFLAKE_WAREHOUSE'),
        database=os.getenv('SNOWFLAKE_DATABASE'),
        role=os.getenv('SNOWFLAKE_ROLE')
    )
    print("Connection Successful")

    # Testing the connection
    cursor = conn.cursor()
    cursor.execute("SELECT CURRENT_VERSION()")
    print(f"Snowflake Version: {cursor.fetchone()[0]}")

except Exception as e:
    print(f"Connection Failed {e}")



### Loading the local files to Snowflake

In [ ]:
local_csv_dir = "../../kaggle_csv_files/"

### Creating a stage to store the kaggle_csv_files
try:
    cursor.execute("USE WAREHOUSE OLIST_WH")
    cursor.execute("USE DATABASE OLIST")
    cursor.execute("USE SCHEMA RAW")
    cursor.execute("CREATE OR REPLACE STAGE olist_raw_stage;")
    print("Stage 'olist_raw_stage' created")

    ## Looping through the folder 
    for filename in os.listdir(local_csv_dir):
        if filename.endswith(".csv"):
            file_path = os.path.join(local_csv_dir, filename)
            print(f"Uploading file {filename}...")

            ## PUT sends the file to stage
            cursor.execute(f"PUT 'file://{file_path}' @olist_raw_stage AUTO_COMPRESS=FALSE OVERWRITE=TRUE;")
    print("\nAll files successfully uploaded to the snowflake stage.")

    ## Verifying the uploaded files
    cursor.execute("LIST @olist_raw_stage;")
    print("Files currently in the stage:")
    for row in cursor.fetchall():
        print(f" - {row[0]}")

except Exception as e:
    print(f"Upload failed: {e}")

### Create tables for the csv files in the RAW schema

In [ ]:
cursor.execute("""
                       CREATE OR REPLACE FILE FORMAT olist_csv_format
                       TYPE = 'CSV'
                       FIELD_DELIMITER = ','
                       PARSE_HEADER = TRUE 
                       FIELD_OPTIONALLY_ENCLOSED_BY = '"'
                       """)

In [ ]:
import os

## Get the list of the files in the stage
cursor.execute("LIST @olist_raw_stage;")
files_in_stage = cursor.fetchall()

try:    
    for row in files_in_stage:
        ## get the internal path
        full_path = row[0]
        filename = full_path.split('/')[-1] #get the filename

        ## Derive the table name from the file
        table_name = filename.replace('olist_', '').replace('_dataset','').split('.')[0].upper()

        print(f"Processing {filename} -> Table: RAW.{table_name}...")

        try:
            
            
            #Use template and Infer-Schema to build the table
            cursor.execute(f"""
                            CREATE OR REPLACE TABLE RAW.{table_name}
                            USING TEMPLATE (
                            SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
                            FROM TABLE (
                                INFER_SCHEMA(
                                LOCATION => '@olist_raw_stage/{filename}',
                                FILE_FORMAT => 'olist_csv_format'
                                )
                                )
                                )
                            """)
            #4. Copy the data from Stage to the new Table
            cursor.execute(f"""
                        COPY INTO RAW.{table_name} FROM @olist_raw_stage/{filename} 
                        FILE_FORMAT = (FORMAT_NAME = 'olist_csv_format')
                        MATCH_BY_COLUMN_NAME = CASE_SENSITIVE
                        PURGE = FALSE 
                        """)
            print(f"Confirmation: RAW.{table_name} loaded successfully.")

        except Exception as e:
            print(f"Failed to load {table_name}: {e}")

finally:
    if 'conn' in locals():
        conn.close()

print("\n--- All ingestion tasks complete ---")



